In [25]:
import sys
import os

sys.path.insert(0, os.path.abspath('../..'))

from core.agents.tools.remote_tool import create_remote_tool

MOCK_METHODS = {
    "get_article": {
        "description": "Get information about a news article",
        "input_types": [str],  # URL
        "mock_response": {
            "title": "News Article",
            "author": "John Doe",
            "published_date": "2025-03-22",
            "content": "This is an example news article content."
        }
    }
}

tool_instance = create_remote_tool(
        name="NewsAPI",
        url="https://example.com/api",
        api_key="test_key",
        mock=True,
        mock_methods=MOCK_METHODS
    )

In [26]:
from dotenv import load_dotenv
import importlib
import os
from langchain_core.messages import AIMessage, BaseMessage, HumanMessage, ToolMessage, SystemMessage
from langchain_ollama import ChatOllama
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode, tools_condition
from typing import Annotated, TypedDict


MODEL = "mistral-nemo"
TEMPERATURE = 0
load_dotenv('../.env', override=True)

False

In [27]:
TOOL_REGISTRY = {
    # 'tools.calculator': ['multiply', 'add', 'divide'],
    # 'tools.wikipedia': ['query'],
    'core.agents.tools.remote_tool': ['remote_tool']
}

def import_function(module_name, function_name):
    """Dynamically imports a function from a module.

    Args:
        module_name: The name of the module (e.g., "my_module").
        function_name: The name of the function to import (e.g., "my_function").

    Returns:
        The imported function, or None if the module or function is not found.
    """
    try:
        module = importlib.import_module(module_name)
        function = getattr(module, function_name)
        return function
    except (ImportError, AttributeError):
        print(f"Error: Could not import function '{function_name}' from module '{module_name}'.")
        return None

tools = [import_function(module, function) for module, functions in TOOL_REGISTRY.items() for function in functions]
# Filter out None values in case import failed
tools = [tool for tool in tools if tool is not None]
if tools:
    print(f"Tools: {[tool.__name__ for tool in tools]}")

Tools: ['remote_tool']


In [28]:
# Only bind tools if we have valid tools
if tools:
    llm = ChatOllama(
        model=MODEL, 
        temperature=TEMPERATURE,
        # base_url="http://host.docker.internal:11434", # if running in the studio
        ).bind_tools([tool for tool in tools])
else:
    llm = ChatOllama(
        model=MODEL, 
        temperature=TEMPERATURE,
        # base_url="http://host.docker.internal:11434", # if running in the studio
        )

class State(TypedDict):
    messages: Annotated[list[BaseMessage], add_messages]
    claim: str
    evidence: list[dict]  # {'name': tool name, 'args': {kwargs}, 'result': str}

In [29]:
from IPython.display import Image, display



with open('prompts/research_agent_system_prompt.txt', 'r') as f:
    sys_msg = SystemMessage(content=f.read())

def preprocessing(state: State):
    """
    Preprocesses state before sending to the assistant for tool routing.
    Currently, this just extracts the claim from the state and sets it as a HumanMessage
    following the SystemMessage
    """
    return {"messages": HumanMessage(content=state['claim'])}

def assistant(state: State) -> State:
    response = llm.invoke(state['messages'])
    return {"messages": response}

def postprocessing(state: State) -> State:
    """
    Scan the message history to extract tool calls and results into tuples:
    (tool_name, tool_args, tool_result) for the 'evidence' list in the state
    """
    
    evidence = []
    for i in range(len(state['messages'])):
        message = state['messages'][i]
        if isinstance(message, AIMessage) and hasattr(message, 'tool_calls'):
            for tool_call in message.tool_calls:
                # Scan later messages for the corresponding ToolMessage
                for j in range(i + 1, len(state['messages'])):
                    next_message = state['messages'][j]
                    if isinstance(next_message, ToolMessage) and next_message.tool_call_id == tool_call['id']:
                        # Found the corresponding ToolMessage
                        evidence.append({
                            'name': tool_call['name'], 
                            'args': tool_call['args'], 
                            'result': next_message.content})
                        break

    return {'evidence': evidence}
    # return state


# Graph
builder = StateGraph(State)

# Define nodes: these do the work
builder.add_node("preprocessing", preprocessing)
builder.add_node("assistant", assistant)
builder.add_node("tools", ToolNode(tools))
builder.add_node("postprocessing", postprocessing)

# Define edges: these determine how the control flow moves
builder.add_edge(START, "preprocessing")
builder.add_edge("preprocessing", "assistant")
builder.add_conditional_edges(
    source="assistant",
    path=tools_condition,
    path_map={'tools': 'tools', '__end__': 'postprocessing'}
)
builder.add_edge("tools", "assistant")
builder.add_edge("postprocessing", END)

react_graph = builder.compile()

# Show
# display(Image(react_graph.get_graph(xray=False).draw_mermaid_png()))

In [30]:
# from langchain_core.messages import HumanMessage, SystemMessage
claim = "1/3 is bigger than 1/4."
# initial_state = {"claim": claim}
# final_state = graph.invoke(initial_state)

messages = [sys_msg]
messages = react_graph.invoke({"messages": messages, "claim": claim})

In [31]:
for m in messages['messages']:
    m.pretty_print()

================================ System Message ================================

You are an **Evidence Retrieval Agent**. Your role is to retrieve supporting or contradictory evidence for a given claim using the tools available to you.  

### **Rules:**  
1. **Do Not Evaluate the Claim** – Your task is to retrieve evidence, not to judge the claim's truth.  
2. **Select and Use Tools Strategically** – Based on the claim, think carefully which tools would help, and construct the correct tool calls to retrieve relevant evidence.  
3. **Return Only Tool Messages** – Your output should consist solely of tool calls with properly structured arguments. Do not generate natural language explanations.  
4. **Handle Ambiguity Thoughtfully** – If the claim is unclear, attempt a best-guess search or request clarification through a tool call if applicable.  
5. **Retrieve Evidence from Multiple Perspectives** – If conflicting evidence exists, return both supporting and contradictory sources.  

### 

In [32]:
from pprint import pprint
pprint(messages)

{'claim': '1/3 is bigger than 1/4.',
 'evidence': [{'args': {'greater': {'a': 0.25, 'b': 0.25}},
               'name': 'calculator',
               'result': 'Error: calculator is not a valid tool, try one of '
                         '[remote_tool].'}],
 'messages': [SystemMessage(content='You are an **Evidence Retrieval Agent**. Your role is to retrieve supporting or contradictory evidence for a given claim using the tools available to you.  \n\n### **Rules:**  \n1. **Do Not Evaluate the Claim** – Your task is to retrieve evidence, not to judge the claim\'s truth.  \n2. **Select and Use Tools Strategically** – Based on the claim, think carefully which tools would help, and construct the correct tool calls to retrieve relevant evidence.  \n3. **Return Only Tool Messages** – Your output should consist solely of tool calls with properly structured arguments. Do not generate natural language explanations.  \n4. **Handle Ambiguity Thoughtfully** – If the claim is unclear, attempt a best

In [33]:
response = react_graph.nodes['assistant'].invoke(
    {"messages": [HumanMessage(content="hi there")]}
)

In [34]:
print(response)

{'messages': AIMessage(content='Hello! How can I assist you today?', additional_kwargs={}, response_metadata={'model': 'mistral-nemo', 'created_at': '2025-03-23T19:31:49.66056Z', 'done': True, 'done_reason': 'stop', 'total_duration': 1052529666, 'load_duration': 13540250, 'prompt_eval_count': 68, 'prompt_eval_duration': 548689750, 'eval_count': 10, 'eval_duration': 489844333, 'message': Message(role='assistant', content='Hello! How can I assist you today?', images=None, tool_calls=None)}, id='run-7fc1d54e-9db1-4e4f-ac7a-296e8be1f5c4-0', usage_metadata={'input_tokens': 68, 'output_tokens': 10, 'total_tokens': 78})}


In [35]:
# Test the remote_tool function directly
from core.agents.tools.remote_tool import remote_tool

# Test with a sample URL
sample_url = "https://example.com/news/article123"
result = remote_tool(sample_url)
print("Direct tool result:")
pprint(result)

Direct tool result:
{'author': 'John Doe',
 'content': 'This is an example news article content.',
 'published_date': '2025-03-22',
 'title': 'News Article'}


In [36]:
# Test using the tool with the research agent
test_claim = "The news article was written by John Doe."
print(f"Testing claim: {test_claim}")

# Reset messages with system message
test_messages = [sys_msg]
# Invoke the graph with the test claim
test_result = react_graph.invoke({"messages": test_messages, "claim": test_claim})

# Print the messages
print("\nMessage exchange:")
for m in test_result['messages']:
    m.pretty_print()

# Print the evidence collected
print("\nEvidence collected:")
pprint(test_result['evidence'])

Testing claim: The news article was written by John Doe.

Message exchange:
================================ System Message ================================

You are an **Evidence Retrieval Agent**. Your role is to retrieve supporting or contradictory evidence for a given claim using the tools available to you.  

### **Rules:**  
1. **Do Not Evaluate the Claim** – Your task is to retrieve evidence, not to judge the claim's truth.  
2. **Select and Use Tools Strategically** – Based on the claim, think carefully which tools would help, and construct the correct tool calls to retrieve relevant evidence.  
3. **Return Only Tool Messages** – Your output should consist solely of tool calls with properly structured arguments. Do not generate natural language explanations.  
4. **Handle Ambiguity Thoughtfully** – If the claim is unclear, attempt a best-guess search or request clarification through a tool call if applicable.  
5. **Retrieve Evidence from Multiple Perspectives** – If conflictin